**DL Final Project**

**2016122015**

**Applied Statistics**

**Sunwoo Kim**

Project : 

***Evaluating the Prediction Power of RNN, LSTM, GRU***



---


** WARNING **

There are some codes that takes huge time(codes of 3.5)

***Please do not Operate Entire RUNTIME in Once***


---



About the Codes 

0. Packages

1. About Data

2. Defining some Functions

3. The Training and Predicting

4. Statistical Model SARIMAX

0. Importing the necessary Packages

In [0]:
## Basic Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set()
from statistics import mean
import math
import warnings
warnings.filterwarnings("ignore")

## The RNN 
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU, Activation, Dropout
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

## The Statistics
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
import statsmodels.api as sm
import itertools
from sklearn.metrics import mean_squared_error

1. Importing and preprocessing the Data

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

In [0]:
## The basic Raw Data
fire = pd.read_csv('/content/gdrive/My Drive/STA_DL/forest_fire.csv', header = None)

train_data = fire[:250]
test_data = fire[250:]

In [0]:
## Producing log train data
## The data that has a value of 0 should get value greater than 1 to get into the log
train_data1 = train_data.copy()
train_data1[train_data1 < 2] = 1 
log_train_data = np.log(train_data1)

In [0]:
## Data Visualizing

Month_1 = []
for i in range(12) : 
  Month_2 = []
  for s in range(19) : 
    data = int(fire.iloc[i+s*12])
    Month_2.append(data)
  Month_1.append(mean(Month_2))

plt.figure(figsize = (16,6)) 
plt.subplot(121) ; plt.plot(fire) ;plt.title('Total Occurence')
plt.subplot(122) ; plt.bar(range(len(Month_1)), Month_1) ; plt.title('Mean Monthly Occurence')
plt.show()
print(fire.shape)

2. Generating The Function

2.1) Function for the Data Generating

In [0]:
## Producing Input and Label of RNN 

def input_generator(x, n) : 
  x1 = np.zeros((int(x.shape[0]-n+1)*n)).reshape(int(x.shape[0]-n+1), n)
  for i in range(int(x.shape[0]-n+1)) : 
    for s in range(n) : 
      x1[i,s] = int(x[i:i+n].iloc[s])
  return(x1)

def y_generator (y,n) : 
  y1 = y[n:]
  return(y1)

In [0]:
## rnn_x, rnn_y = The Raw Training Data Set
## log_rnn_x, log_rnn_y = The log Training Data Set

rnn_x1 = input_generator(train_data, 12)[:-1]
rnn_x = rnn_x1.reshape(rnn_x1.shape[0], 1, rnn_x1.shape[1])
rnn_y = y_generator(train_data, 12)

rnn_x2 = input_generator(log_train_data, 12)[:-1]
log_rnn_x = rnn_x2.reshape(rnn_x2.shape[0], 1, rnn_x2.shape[1])
log_rnn_y = y_generator(log_train_data, 12)

In [0]:
## Here it Produces the Prediction Values
## This automatically takes the previous predicted value to the input of next prediction steps
## The final Prediction Value of '18.01 to '19.12 comes out as a numpy array

def prediction_generator(model) : 
  final_v = []
  x1 = np.concatenate([rnn_x[-1][0][1:],[int(rnn_y.iloc[-1])]])
  for i in range(24) : 
    new_v = int(model.predict(x1.reshape(1,1,12)))
    final_v.append(new_v)
    x1 = np.concatenate([x1[1:], np.array([new_v])])
  return(final_v)

def log_prediction_generator(model) : 
  final_v = []
  x1 = np.concatenate([log_rnn_x[-1][0][1:],[int(log_rnn_y.iloc[-1])]])
  for i in range(24) : 
    new_v = int(model.predict(x1.reshape(1,1,12)))
    final_v.append(new_v)
    x1 = np.concatenate([x1[1:], np.array([new_v])])
  return(final_v)

2.2) Funtion for the Neural Network Model

In [0]:
def single_layer_SRNN (n, d) : 
  model = Sequential()
  model.add(SimpleRNN(n, input_shape = (1,12), return_sequences = False, activation = 'relu'))
  model.add(Dropout(d))
  model.add(Dense(1, activation = 'relu'))
  model.compile(loss = 'mse', optimizer = 'adam')
  return model

In [0]:
def single_layer_LSTM (n, d) : 
  model = Sequential()
  model.add(LSTM(n, input_shape = (1,12), return_sequences = False, activation = 'relu'))
  model.add(Dropout(d))
  model.add(Dense(1, activation = 'relu'))
  model.compile(loss = 'mse', optimizer = 'adam')
  return model

In [0]:
def single_layer_GRU (n, d) : 
  model = Sequential()
  model.add(GRU(n, input_shape = (1,12), return_sequences = False, activation = 'relu'))
  model.add(Dropout(d))
  model.add(Dense(1, activation = 'relu'))
  model.compile(loss = 'mse', optimizer = 'adam')
  return model

In [0]:
def double_layer_SRNN(n1,n2,d) :
  model = Sequential()
  model.add(SimpleRNN(n1, input_shape =(1,12), return_sequences =  True, activation = 'relu'))
  model.add(Dropout(d))
  model.add(SimpleRNN(n2, return_sequences = False,activation = 'relu'))
  model.add(Dropout(d))
  model.add(Dense(1, activation = 'relu'))
  model.compile(loss = 'mse', optimizer = 'adam')
  return model

In [0]:
def double_layer_LSTM(n1, n2, d) :
  model = Sequential()
  model.add(LSTM(n1, input_shape =(1,12), return_sequences =  True, activation = 'relu'))
  model.add(Dropout(d))
  model.add(LSTM(n2, return_sequences = False,activation = 'relu'))
  model.add(Dropout(d))
  model.add(Dense(1, activation = 'relu'))
  model.compile(loss = 'mse', optimizer = 'adam')
  return model

In [0]:
def double_layer_GRU(n1, n2, d) :
  model = Sequential()
  model.add(GRU(n1, input_shape =(1,12), return_sequences =  True, activation = 'relu'))
  model.add(Dropout(d))
  model.add(GRU(n2, return_sequences = False,activation = 'relu'))
  model.add(Dropout(d))
  model.add(Dense(1, activation = 'relu'))
  model.compile(loss = 'mse', optimizer = 'adam')
  return model

In [0]:
## To obtain the optimal nodes number, We can Get the Result Table
def single_layer_result(m,a,b) :
  res_t = pd.DataFrame(data = {'Node_Number' : [0], 'Mean_Err' : [0], 'Pred_Err' : [0]}, columns = ['Node_Number', 'Mean_Err', 'Pred_Err'])
  for i in range(a,b) : 
    model = m(i, 0.3)
    hist = model.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
    mean_loss = mean(hist.history['loss'][15:])
    predy = prediction_generator(model)
    pred_loss = mean_squared_error(predy, test_data)
    Partial = pd.DataFrame(data = {'Node_Number' : [i], 'Mean_Err' : [mean_loss], 'Pred_Err' : [pred_loss]}, columns = ['Node_Number', 'Mean_Err', 'Pred_Err'])
    res_t = pd.concat([res_t, Partial], axis =0, join = 'outer')
  return res_t 

In [0]:
def double_layer_result(m,a,b,c,d) :
  res_t = pd.DataFrame(data = {'Node_Number1' : [0], 'Node_Number2' : [0], 'Mean_Err' : [0], 'Pred_Err' : [0]}, columns = ['Node_Number1', 'Node_Number2','Mean_Err', 'Pred_Err'])
  for i in range(a,b) :
    for s in range(c,d) :  
      model = m(i, s, 0.3)
      hist = model.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
      mean_loss = mean(hist.history['loss'][15:])
      predy = prediction_generator(model)
      pred_loss = mean_squared_error(predy, test_data)
      Partial = pd.DataFrame(data = {'Node_Number1' : [i], 'Node_Number2' : [s], 'Mean_Err' : [mean_loss], 'Pred_Err' : [pred_loss]}, columns = ['Node_Number1', 'Node_Number2','Mean_Err', 'Pred_Err'])
      res_t = pd.concat([res_t, Partial], axis =0, join = 'outer')
  return res_t 

In [0]:
## Proof for the importance of batch_Selection
## That specific nodes doesn't matter, but the range does exist
def Simulation_T_S(m,a,b) :
  res_t = pd.DataFrame(data = {'Node_Number' : [0], 'Mean_Err' : [0], 'Pred_Err' : [0]}, columns = ['Node_Number', 'Mean_Err', 'Pred_Err'])
  for i in range(a,b) : 
    e_table = []
    p_table = []
    for s in range(30) :
      model = m(i, 0.3)
      hist = model.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
      mean_loss = mean(hist.history['loss'][15:])
      predy = prediction_generator(model)
      pred_loss = mean_squared_error(predy, test_data)
      e_table.append(mean_loss)
      p_table.append(pred_loss)
    M_L = mean(e_table)
    P_L = mean(p_table)
    Partial = pd.DataFrame(data = {'Node_Number' : [i], 'Mean_Err' : [M_L], 'Pred_Err' : [P_L]}, columns = ['Node_Number', 'Mean_Err', 'Pred_Err'])
    res_t = pd.concat([res_t, Partial], axis =0, join = 'outer')
  return res_t 

3. Getting the Training and Prediction Result

3.1) Log Data vs Raw Data

In [0]:
model0_0 = single_layer_SRNN(50, 0.3)
model0_0.summary()
hist1 = model0_0.fit(log_rnn_x, log_rnn_y, batch_size = 40, epochs = 25)
plt.plot(hist1.history['loss'])
plt.title('Training Loss')
plt.show()
predy = np.exp(log_prediction_generator(model1))
pred_loss = mean_squared_error(predy, test_data)
print(pred_loss)
x1 = np.arange(0, 24)
plt.plot( x1, predy, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('log Data')
plt.show()

In [0]:
model0_1 = single_layer_SRNN(50, 0.3)
model0_1.summary()
hist1 = model0_1.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
plt.plot(hist1.history['loss'])
plt.title('Training Loss')
plt.show()
predy = prediction_generator(model0_1)
pred_loss = mean_squared_error(predy, test_data)
print(pred_loss)
x1 = np.arange(0, 24)
plt.plot( x1, predy, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('log Data')
plt.show()

3.1 Result. The Raw data gives better Result

3.2) The Simple RNN

In [0]:
## Single Layer Simple RNN
model1 = single_layer_SRNN(40, 0.3)
model1.summary()
hist1 = model1.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
plt.plot(hist1.history['loss'])
plt.title('Training Loss')
plt.show()
predy1 = prediction_generator(model1)
pred_loss1 = mean_squared_error(predy1, test_data)
print(pred_loss1)
x1 = np.arange(0, 24)
plt.plot( x1, predy1, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('Prediction Err')
plt.show()

In [0]:
## Double Layer Simple RNN
model2 = double_layer_SRNN(20, 60, 0.2)
model2.summary()
hist2 = model2.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
plt.plot(hist2.history['loss'])
plt.title('Training Loss')
plt.show()
predy2 = prediction_generator(model2)
pred_loss2 = mean_squared_error(predy2, test_data)
print(pred_loss2)
x1 = np.arange(0, 24)
plt.plot( x1, predy2, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('Prediction Err')
plt.show()

3.3) The LSTM

In [0]:
## Single Layer LSTM
model3 = single_layer_LSTM(26, 0.3)
model3.summary()
hist3 = model3.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
plt.plot(hist3.history['loss'])
plt.title('Training Loss')
plt.show()
predy3 = prediction_generator(model3)
pred_loss3 = mean_squared_error(predy3, test_data)
print(pred_loss3)
x1 = np.arange(0, 24)
plt.plot( x1, predy3, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('Prediction Err')
plt.show()

In [0]:
## Double Layer LSTM
model4 = double_layer_LSTM(40, 40, 0.2)
model4.summary()
hist4 = model4.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
plt.plot(hist4.history['loss'])
plt.title('Training Loss')
plt.show()
predy4 = prediction_generator(model4)
pred_loss4 = mean_squared_error(predy4, test_data)
print(pred_loss4)
x1 = np.arange(0, 24)
plt.plot( x1, predy4, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('Prediction Err')
plt.show()

3.4) The GRU

In [0]:
## Single Layer GRU
model5 = single_layer_GRU(70, 0.3)
model5.summary()
hist5 = model5.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
plt.plot(hist5.history['loss'])
plt.title('Training Loss')
plt.show()
predy5 = prediction_generator(model5)
pred_loss5 = mean_squared_error(predy5, test_data)
print(pred_loss5)
x1 = np.arange(0, 24)
plt.plot( x1, predy5, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('Prediction Err')
plt.show()

In [0]:
## Double Layer GRU
model6 = double_layer_LSTM(60, 60, 0.2)
model6.summary()
hist6 = model6.fit(rnn_x, rnn_y, batch_size = 40, epochs = 25)
plt.plot(hist6.history['loss'])
plt.title('Training Loss')
plt.show()
predy6 = prediction_generator(model6)
pred_loss6 = mean_squared_error(predy6, test_data)
print(pred_loss6)
x1 = np.arange(0, 24)
plt.plot( x1, predy6, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('Prediction Err')
plt.show()

3.5) Trying with Various nodes number

** WARNING **

Following 3.5 codes takes Huge Time 

In [0]:
## Getting the Result for the range of nodes number 20 to 100
Optimal_single_SRNN = single_layer_result(single_layer_SRNN, 20, 100)[1:]
Optimal_single_LSTM = single_layer_result(single_layer_LSTM, 20, 100)[1:]
Optimal_single_GRU = single_layer_result(single_layer_GRU, 20, 100)[1:]

In [0]:
## Getting the Result for the range of nodes number 20 to 60 or 40 to 80
Optimal_double_SRNN = double_layer_result(double_layer_SRNN, 20, 60, 20, 60)[1:]
Optimal_double_LSTM = double_layer_result(double_layer_LSTM, 20, 60, 20, 60)[1:]
Optimal_double_GRU = double_layer_result(double_layer_GRU, 40, 80, 40, 80)[1:]

In [0]:
## Simulation for the same nodes number various times 
## Simulation for the nodes from a to b for 30 times each
## This is only for the Single Layer Model
Simulation = Simulation_T_S(single_layer_LSTM,20,40)

In [0]:
## Visualizing the Simulation Result

x1 = np.arange(20, 40)
plt.plot(x1, Simulation.Mean_Err[1:])
plt.xlabel('Node Number')
plt.title('Mean Training Loss 10 times Simulation')
plt.show()

x2 = np.arange(20, 40)
plt.plot(x2, Simulation.Pred_Err[1:])
plt.xlabel('Node Number')
plt.title('Mean Prediction Loss 10 times Simulation')

4) About Statistical Model

Using SARIMAX 

In [0]:
## Drawing ACF, PACF for log_train_data
print(plot_acf(log_train_data, lags = 40))
print(plot_pacf(log_train_data, lags = 40))

In [0]:
decompose = sm.tsa.seasonal_decompose(log_train_data, model='add', freq = 12)
fig = decompose.plot()
plt.show()

In [0]:
## Searching for Optimal p, d, q value for the SARIMAX

p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

d_pdq = []
for i in range(27) :
  if pdq[i][1] == 1 :
    d_pdq.append(pdq[i])

s_pdq = []
for i in range(27) :
  if seasonal_pdq[i][1] == 1 :
    s_pdq.append(seasonal_pdq[i])

final = pd.DataFrame(data = {'De_Seasonal' : [(0,0,0)], 'Seasonal' : [(0,0,0)], 'AIC' : [10000]}, columns = ['De_Seasonal', 'Seasonal', 'AIC'])
for param in d_pdq:
    for param_seasonal in s_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(log_train_data,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            Partial = pd.DataFrame(data = {'De_Seasonal' : [param], 'Seasonal' : [param_seasonal], 'AIC' : results.aic}, columns = ['De_Seasonal', 'Seasonal', 'AIC'])
            final = pd.concat([final, Partial], axis =0, join = 'outer')
        except: 
            continue
final

In [0]:
## The Result for the SARIMAX

mod = sm.tsa.statespace.SARIMAX(
                                #train_data,
                                log_train_data,
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
res = mod.fit()
print(res.summary())

res.plot_diagnostics(figsize = (18,8))
plt.show()

pred = res.get_prediction(start = 1, end = 24, dynamic = False)
#y_pred = pred.predicted_mean
y_pred = np.exp(pred.predicted_mean)
print('Test MSE is', mean_squared_error(test_data[0], y_pred))

x1 = np.arange(0, 24)
plt.plot( x1, y_pred, x1, test_data, 'r-')
plt.legend(('Pred', 'True'))
plt.title('True Data and Predicted Data')